In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle
import matplotlib.pyplot as plt

In [2]:
from process import gen_second_data

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [4]:
second_train = "20190721_second_train.pickle"
second_val = "20190721_second_val.pickle"
second_test = "20190721_second_test.pickle"
first_train = "20190721_dist_ang_ori_bond_cos_train_{}.pickle"
first_test = "20190721_dist_ang_ori_bond_cos_test_{}.pickle"

In [5]:
df_val_temp = pd.read_pickle(OUTPUT + second_val)
df_train_temp = pd.read_pickle(OUTPUT + second_train)

In [6]:
df_feat = pd.DataFrame()
bond_types = ['1JHN', '1JHC', '2JHH', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN']
for b in bond_types:
    df_bond = pd.read_pickle(OUTPUT + first_train.format(b)).fillna(0)
    df_feat = pd.concat([df_feat, df_bond], axis=0)

In [7]:
df_feat['type'].value_counts()

3JHC    1510379
2JHC    1140674
1JHC     709416
3JHH     590611
2JHH     378036
3JHN     166415
2JHN     119253
1JHN      43363
Name: type, dtype: int64

In [8]:
df_train_temp['type'].value_counts()

3JHH    557974
Name: type, dtype: int64

In [11]:
df_train = pd.merge(df_feat, df_train_temp, \
                    left_on=['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type','scalar_coupling_constant'],
                    right_on=['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type','scalar_coupling_constant'])

In [12]:
df_val = pd.merge(df_feat, df_val_temp, \
                    left_on=['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type','scalar_coupling_constant'],
                    right_on=['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type','scalar_coupling_constant'])

In [13]:
rounds = 100000
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : {'l1'},
        'num_leaves' : 63,
        'learning_rate' : 0.005,
        'feature_fraction' : 0.5,
        'bagging_fraction' : 0.5,
        'bagging_freq': 5,
    'seed':0
}

In [14]:
def type_score(y_val, y_pred):
    return np.log(sum(np.abs(y_val- y_pred)) / len(y_val))

In [17]:
y_train.shape

(0,)

In [16]:
for b in bond_types:
    print('-'*10, b, '-'*10)
    
    y_train = df_train.query('type=="{}"'.format(b))['scalar_coupling_constant'].values
    X_train = df_train.query('type=="{}"'.format(b))[df_train.columns[6:]].values

    y_val = df_val.query('type=="{}"'.format(b))['scalar_coupling_constant'].values
    X_val = df_val.query('type=="{}"'.format(b))[df_val.columns[6:]].values
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    
    start = time.time()
    gbm = lgb.train(params,
            lgb_train,
            num_boost_round=rounds,
            verbose_eval=0,
            valid_sets=lgb_eval,
            early_stopping_rounds=30)
    
    y_val_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
    print(gbm.best_iteration)
    print("score first model: %.2f" %  (type_score(y_val, X_val[:, -98])))
    print("score second model: %.2f" %  (type_score(y_val, y_val_pred)))
    plt.scatter(y_val, X_val[:, -98])
    plt.scatter(y_val, y_val_pred)
    plt.show()
    elapsed_time = time.time() - start
    print ("elapsed_time:%.2f" % elapsed_time + "[sec]")


---------- 1JHN ----------


LightGBMError: Check failed: num_data > 0 at /__w/1/s/python-package/compile/src/io/dataset.cpp, line 27 .


In [ ]:
df_imp = pd.Series(gbm.feature_importance(), index=df_train.query('type=="{}"'.format(bond))[df_train.columns[6:]].columns)

In [ ]:
df_imp